## Spotify Song Analysis Project:

#### Project Purpose:

#### This project will collect feature data from the Spotify API from my highest streamed songs songs and the highest streamed songs of friends with different musical preferences and compute the averages of each feature. 

#### With that data, I will then analyze randomly selected playlists to find songs that are close to my averages. 

#### To test the effectiveness of the Spotify recommendations algorithm and how well it works for people with similar and different music tastes, I will then stream/send the songs closest to me and my friend's averages.

#### This project is mostly exploratory because I think the ability to suggest new songs from the Billboard top 100 or from certain genres and send it through email is ultimately the goal of the project.

#### It is possible to create an airflow process that searches and finds spotify urls for playlists and looks for songs that match certain parameters and sends it to your phone

#### You can also use 

##### Importing all necessary dependencies

In [ ]:
import os
import json
from pathlib import Path
import pandas as pd
from decouple import config
import sqlite3
import spotipy
import spotipy.oauth2 as oauth2
import boto3

##### Data Extraction Process

In [ ]:
%load_ext dotenv
%dotenv
import os
from datetime import date
today=date.today()

from dotenv import load_dotenv
CLIENT_ID=load_dotenv('CLIENT_ID')
CLIENT_SECRET=load_dotenv('CLIENT_SECRET')
USERNAME=load_dotenv('USERNAME')


##### My playlist consists of all 30 of my recently most played songs and 70 randomly selected songs from 'Your Top Songs 2022'

##### Playlists from the three selected friends consists of their top current songs and all-time favorite songs.

##### The randomly selected songs were selected by randomly selecting album search keywords, then randomly selecting the search result (ex: album #21 out of 5987 results) and finally selecting a random song within the album (ex: song #31 out of 672).

##### After selecting the songs, the song links are then collected as strings and passed into respective lists according to how the song was selected.<br><br>

##### Loading all respective tracks as lists:

In [ ]:
my_tracks=['https://open.spotify.com/track/7l3ZNXBVVdhBz13Vo0kbLz?si=102c62f639d44a0d','https://open.spotify.com/track/2dXK1KxlHZSyTXO914O0SP?si=ef85ed0713924656',
           'https://open.spotify.com/track/2MAaiV0bbeq4FpSwPe2rR8?si=88e4f8c55f16465a','https://open.spotify.com/track/5JS1gz9zP4GTy0LZYvSeUK?si=f2c8ab7a5dc24a63',
           'https://open.spotify.com/track/67mtN6BlsJR0LVyzvNZnGt?si=a3f1cd3630bc46be','https://open.spotify.com/track/4rpqJYjM1uvAz4F3PPlL99?si=2c90de074bfb4d6b',
           'https://open.spotify.com/track/7IwURvEfVcdxUCjLKUu6sv?si=8c20183267334fee','https://open.spotify.com/track/3mTpegrOwRn0oJjv4TSbEE?si=63f847ecd3574d60',
           'https://open.spotify.com/track/6Ep56mDirnv1ORaxN061KY?si=9cee97003adb4eaa','https://open.spotify.com/track/1l7xvA5XajmKQDo1h8Vc3K?si=5fa9c818e3aa4f0f',
           'https://open.spotify.com/track/1lXBO9MA1ZnyQSaMPS0Ipo?si=484fdac03b494107','https://open.spotify.com/track/0eEXcw3JLVXcRxYrVYMy68?si=d820dd56036641ea',
           'https://open.spotify.com/track/6Jk15hKKyysyF2iCNPTZBY?si=7402b78a980c4f4d','https://open.spotify.com/track/4QjtpTswV0kbzqoE60XDpk?si=d67c167332004aae',
           'https://open.spotify.com/track/4anqYZt4APNlFwQixpbdZR?si=dc1796f6bec74344','https://open.spotify.com/track/15HgR9Ht4obJOcMG4wregp?si=99d5ba3612844b62',
           'https://open.spotify.com/track/4jv7YxKO7S6M3Qj4Dun8sZ?si=2c08434a2daa4549','https://open.spotify.com/track/4L0aj6GxRyDoPuCZq7qsIc?si=7a74e2b5fab34f2a',
           'https://open.spotify.com/track/07H3Mpy3KcHKCe7renjFSi?si=9bdaf67899de4554','https://open.spotify.com/track/27GmP9AWRs744SzKcpJsTZ?si=31d56ceb49674387',
           'https://open.spotify.com/track/2mu2oL9gdgHGtlg9poer2u?si=c1f99d48f1a649e9','https://open.spotify.com/track/4TGHOWSNAwUXn5onDQr0P1?si=b11ea9bf3499463f',
           'https://open.spotify.com/track/4UvulgTFTqbyzSeJI2vfYq?si=efe5b63d92144e48','https://open.spotify.com/track/557DLyFeg06jCV2cpT005C?si=b8bb89be06fc4172',
           'https://open.spotify.com/track/7CH99b2i1TXS5P8UUyWtnM?si=fd68c29cadb54667','https://open.spotify.com/track/5FG7Tl93LdH117jEKYl3Cm?si=eb53b9a3da9641e7',
           'https://open.spotify.com/track/06GhyyKkQrMO73qVArgQVH?si=b2201900ac7c4462','https://open.spotify.com/track/3uJ0spXcRqOO8PCGAnjSeQ?si=d393c4ea76864f1f',
           'https://open.spotify.com/track/7Jayh7T1793wFTGOWD4BNw?si=dafa026dcbe54384','https://open.spotify.com/track/3GhW5KDkzKGOBNawGWUuaO?si=008e342084f9413a',           
           'https://open.spotify.com/track/0pSaDwrMLM7F82LWJg9v6s?si=e135528f01fd40cd','https://open.spotify.com/track/6z1nMCq1HJSZ78u2jYtrDT?si=b31fa5e7118842c6',
           'https://open.spotify.com/track/4qcMCzJnf02hJBUw9kyl0D?si=a51af975b7c54b0f','https://open.spotify.com/track/1OoPhcM78evXIxjmSzCnmf?si=1c091e23fd99422a',
           'https://open.spotify.com/track/7K3eUCkR3dEHGw1awTRPB9?si=b93f8bbcc4244681','https://open.spotify.com/track/4kuPeLqTPXDkPB2injreCw?si=9ced21f8d12e4c56',
           'https://open.spotify.com/track/5Ea6VaQFplATEu2biAQdlB?si=3ce5054bcc614314','https://open.spotify.com/track/4pCEIQ6wBVuaJdER5bALtO?si=8c17b9ed0a29473a',
           'https://open.spotify.com/track/5bgbb8KMAZreVHzh3bRPS5?si=de4bc905bd6e490e','https://open.spotify.com/track/5DS9LiyEdw2zY8bM6kjjgM?si=ecf419472cc4474c',
           'https://open.spotify.com/track/0k7wmahjkn389wAZdz19Cv?si=6b9d551c9844496d','https://open.spotify.com/track/2cny5OSTr2976U3o018GoV?si=86a52a647e23406f',
           'https://open.spotify.com/track/1xMvdOFQaql3EBkHmuh2TK?si=8368afba65c54cea','https://open.spotify.com/track/5oady7OSE7dH7LHiqmb3Cr?si=b7ed36dd3f104410',
           'https://open.spotify.com/track/0uxSUdBrJy9Un0EYoBowng?si=163d08940ed647bb','https://open.spotify.com/track/6P5ulGKtC4x6RnFbzfpq8O?si=f1dc103db0834836',
           'https://open.spotify.com/track/2FK1qmw8XNBqUe5CcgTVnE?si=35fda2b4ee954ece','https://open.spotify.com/track/6JbrcjoMVBBNZllVOiHBcU?si=f94f9a8a5f92402f',
           'https://open.spotify.com/track/54TqtV4TWMvM76nJYzKzF3?si=909d7831b3ea4e64','https://open.spotify.com/track/58dSdjfEYNSxte1aNVxuNf?si=9730665c6d314a19',
           'https://open.spotify.com/track/0GaBIpyHvytM1UBYmqXu08?si=c4f11a4a1333487b','https://open.spotify.com/track/2mu2oL9gdgHGtlg9poer2u?si=a843eb90e3824b94',
           'https://open.spotify.com/track/0BFGDkrXAneskCJdudSCro?si=d8f3bfd2fed24a76','https://open.spotify.com/track/3O8NlPh2LByMU9lSRSHedm?si=eaa496507b044033',
           'https://open.spotify.com/track/7hav1MHURQf5SAoPk0egqh?si=b3ea79f353de4a9f','https://open.spotify.com/track/5c549LWiKCWGvcDsZwMN6v?si=ef4d0f6530dc4ab3',
           'https://open.spotify.com/track/01tNpFZF7dO0dXW8YwQxSZ?si=85df0364b4e545a1','https://open.spotify.com/track/3fBbfrWZUuWWrMQXFISr7N?si=be4c663d0ccf43cd',
           'https://open.spotify.com/track/2Ozc0me9PV5vlt8cokwdvI?si=134f5418d7e442f6','https://open.spotify.com/track/3XstzgzP0rp3bzElEnRVHv?si=65b4922eefa14c5b',
           'https://open.spotify.com/track/56uXDJRCuoS7abX3SkzHKQ?si=c9375284a4924b4f','https://open.spotify.com/track/0PXukVbYpvz40KcEFKnIw7?si=9c68a82d865447c8',
           'https://open.spotify.com/track/2Iv3GHM3xFKIPbYSnhJAXO?si=f05fe408fd6f4ca4','https://open.spotify.com/track/6rTInqW3YECMkQsBEHw4sd?si=537fc04c8f444109',
           'https://open.spotify.com/track/5FqaXHqw79ILxvJUKvOzxc?si=5b8ee8de0d7d4e11','https://open.spotify.com/track/0zWbYGQq5O1KQd0iqzgjP6?si=6a37516193324e41',
           'https://open.spotify.com/track/7qfO0IGkZ1a2OnUOkMoR4D?si=0cac7e1ac43d4c69','https://open.spotify.com/track/7E4lZCS6ho36CbhimQklnm?si=351110063e314f99',
           'https://open.spotify.com/track/5iZ5wjTTxSIv8Cfrhj6Jla?si=be9c46abf5704f8e','https://open.spotify.com/track/65OVbaJR5O1RmwOQx0875b?si=2bcb6cced23a4c24',
           'https://open.spotify.com/track/6u3CPnFMKANYgfdiifFOiJ?si=f4511915a62a45dd','https://open.spotify.com/track/3HVJbdz0V402U5YggQK4Ao?si=6712d5602290496b',
           'https://open.spotify.com/track/5ctk3VJoHoURSiwkCJn2bf?si=578b7234efa14fb9','https://open.spotify.com/track/5jUvv1aTcNTgTOsAGlJojn?si=2bd3b01ae4ea464b',
           'https://open.spotify.com/track/3ketN3dth18vSJ1T3HIztN?si=362b09fef70b434b','https://open.spotify.com/track/0lk5tzmaJWVAv5GBvpm3xu?si=1f35ba0e8ee0458a',
           'https://open.spotify.com/track/14uUTfHIwIp587cz13UI5A?si=bc5b4d6172ef4106','https://open.spotify.com/track/2VkusCBEW3EtnXdlZ3V3U5?si=56dd920bdc2f4fb3',
           'https://open.spotify.com/track/7fE8bDKSdxLpgqMr701zws?si=c4ef786003974534','https://open.spotify.com/track/1YDebgap5OFwXEBG18NJvy?si=a3951ec676cf4ac1',
           'https://open.spotify.com/track/2zYs4BonN2ydkbrRk333SN?si=978fd87587ce44b0','https://open.spotify.com/track/15hJmqqEtASVXl6sM7i4UF?si=634ad856066d4dc0',
           'https://open.spotify.com/track/4VcPtoOpK3oBMDZJ9Ptp9f?si=f9a88866d9f34b24','https://open.spotify.com/track/2otVC3BB5KkJNNySWdsDlj?si=ac7c9f3dfd1a4158',
           'https://open.spotify.com/track/0DJ6d7MB8BXfGNKS5SuWJr?si=4d8989b8abcb43b6','https://open.spotify.com/track/4t2HbYiaqvoPsbSrOB0LqS?si=c3967d3c4aa3475d',
           'https://open.spotify.com/track/3n00wtGRMI9Ue45UOPut1H?si=50c3bd32ed2a4f4a','https://open.spotify.com/track/2xqzJOGqkXCTwNOxfIoxUt?si=ff7a5d7e61974938',
           'https://open.spotify.com/track/1zwZ26A7OA5wTrA3FejCLL?si=b01da35a0e1d4b14','https://open.spotify.com/track/6jFfziQ3596GTUE3cZ9v8b?si=91fd6e2a5221437d',
           'https://open.spotify.com/track/3XOalgusokruzA5ZBA2Qcb?si=c3ac0d84db7a46a2','https://open.spotify.com/track/6XHVuErjQ4XNm6nDPVCxVX?si=9021ddb0f1c54889',
           'https://open.spotify.com/track/2GUJWmiDlSwey0GrLCX1gK?si=63346333d5b848b7','https://open.spotify.com/track/62N0RkCCs4XQXk7hgnpOXo?si=702c825c90a2492a',
           'https://open.spotify.com/track/0vwOWwl785S9EU1wtbeLLJ?si=b419f7f8042947ea','https://open.spotify.com/track/40mjsnRjCpycdUw3xhS20g?si=ba338564a4c9498b',
           'https://open.spotify.com/track/4cAbIFjTYcurfQP6EBgf6l?si=c72ff21769214419','https://open.spotify.com/track/05KOgYg8PGeJyyWBPi5ja8?si=d1ce083dc93b4804',
           'https://open.spotify.com/track/64BafeBQMbpUgMz492JVsd?si=b02e244702e84c1e','https://open.spotify.com/track/6SN1H9p6A0UbsNpfkxEEX6?si=1dd44a8b31e1443e']

first_friend_tracks=['https://open.spotify.com/track/6LlQBOQweWj8N5TK4S2HtH?si=4db288a1b9494caf','https://open.spotify.com/track/4tsjduLnwbsFTEgmx0Vmtv?si=d17cabde3dca4ab5',
                     'https://open.spotify.com/track/5qVhJt2g7DIprSiIg5VhpB?si=8193a8a671784c8c','https://open.spotify.com/track/2UzQdfooGVIeGjqcUMOBWY?si=bbdde5288c0c4299',
                     'https://open.spotify.com/track/60xaS8mYBKUW4VQQ666N0T?si=92621979e9ea44e1','https://open.spotify.com/track/3ZLyt2ndLFBh148XRYjYYZ?si=7711ddf6186a4832',
                     'https://open.spotify.com/track/0jx8zY5JQsS4YEQcfkoc5C?si=7414c0b448134770','https://open.spotify.com/track/5IdQEHgtmj9th3OkfQKhf8?si=1eacedf022584e28',
                     'https://open.spotify.com/track/0qy5D3OJre7SPJNMOL9I71?si=4495478410ed40d9','https://open.spotify.com/track/0v9Wz8o0BT8DU38R4ddjeH?si=b7cbdf9c5eaf419b',
                     'https://open.spotify.com/track/2AgnbuzvnfPvq1sAYNB4pj?si=52d5cc644ac346b6','https://open.spotify.com/track/6AoYWCzsDls6c61Zp5qTuU?si=15aa3a922ff24302',
                     'https://open.spotify.com/track/0qrWWx3CcEjT2BUvrtp1ko?si=f8ea6ce71e32422c','https://open.spotify.com/track/530J9GupahE2O4a1iVkQxB?si=d03553918c4f4970',
                     'https://open.spotify.com/track/4Zf351SuPuXo8w6IIMsxev?si=4f7f290dcb044905','https://open.spotify.com/track/6fTdcGsjxlAD9PSkoPaLMX?si=ef60912500cd4be8',
                     'https://open.spotify.com/track/1CoAa7UEqFlO9qIAj2DV88?si=7434b511ba054d73','https://open.spotify.com/track/7jgT6Q0rfinCKLCS7YxrB2?si=27254c83b85d475a',
                     'https://open.spotify.com/track/2EEeOnHehOozLq4aS0n6SL?si=9cc683d55cee4e42','https://open.spotify.com/track/2qtoRFCOEL1gRn5q9DJC7F?si=73b06d78ee384572',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','',
                     '','']

second_friend_tracks=['','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','',
                      '','']
third_friend_tracks=['','','','','','','','','','','']
randomly_selected_tracks=['','','','','','','','','','','']
total_tracks_list=[my_tracks,first_friend_tracks,second_friend_tracks,third_friend_tracks,randomly_selected_tracks]

##### Song features to analyze:

In [ ]:
acousticness=[] 
analysis_url=[] 
danceability=[]
duration_ms=[]
energy=[]
id=[]  #Second Primary Key
instrumentalness=[] 
key=[]
liveness=[]
loudness=[]
mode=[]
speechiness=[] 
tempo=[]
time_signature=[] 
track_href=[]
type=[]
uri=[] #First Primary Key
valence=[] 

##### Spotify Data Extraction Process:

In [ ]:
from numba import jit
%load_ext dotenv
%dotenv

@jit(nopython=True)
def spotify_data_extractor():
    try:
        artist_id=str(input(f'Enter the Spotify Artist ID:'))
        auth=oauth2.SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
        token=auth.get_access_token(as_dict=False)
        sp=spotipy.Spotify(auth=token)
    
        for list in total_tracks_list:
            spotify_data=sp.audio_features(tracks=list)   
        
            with open('output.json', 'w') as api_output_file:
                json.dump(spotify_data,api_output_file,sort_keys=True)
        
            with open('output.json', 'r', encoding='utf-8') as api_output_file:
                imported_output_file = json.loads(api_output_file.read())

            for song in imported_output_file["id"]:
                album_name.append(song["name"])
                artist_name.append(song["artists"][0]["name"])
                release_date.append(song["release_date"])
                total_tracks.append(song["total_tracks"])
                type.append(song["type"])
                uri.append(song["uri"])

                spotify_data_dict= { 

                    "id": id,
                    "uri": uri,
                    "acousticness": acousticness,
                    "analysis_url": analysis_url,
                    "danceability": danceability,
                    "duration_ms": duration_ms,
                    "energy": energy,
                    "instrumentalness": instrumentalness,
                    "key": key,
                    "liveness": liveness,
                    "mode": mode,
                    "speechiness": speechiness,
                    "tempo": tempo,
                    "time_signature": time_signature,
                    "track_href": track_href,
                    "type": type,
                    "valence": valence
                }
                parsed_spotify_data=pd.DataFrame(spotify_data_dict, 
                                                 columns=["id","uri","acousticness","analysis_url",
                                                          "danceability","duration_ms","energy","instrumentalness",
                                                          "key","liveness","mode","speechiness","tempo",
                                                          "time_signature","track_href","type","valence",
                                                          ])
                print(f'Data extraction process completed')

    except Exception as e:
        print(f'Data extraction process could not finish. Please try again.')
        print(e)

    spotify_data_extractor()

##### Data Transformation Process:

In [ ]:
import os

##### Data Loading Process:

In [ ]:
import os

##### Data Analysis:

In [ ]:
spotify_data.agg(function=)